In [1]:
%load_ext autotime

In [ ]:
import json
import importlib
import pandas as pd
from pathlib import Path

from wb_nlp.interfaces import mongodb
from wb_nlp.types import metadata as meta_type
from wb_nlp.types import metadata_enums
from wb_nlp import dir_manager

importlib.reload(meta_type)
importlib.reload(metadata_enums)

# World Bank Metadata

Load the metadata from the API scraper notebook: `wb_nlp/notebooks/scrapers/wb_api_scraper_latest.ipynb`

In [ ]:
# # fname = f"wb_metadata-{datetime.now()}.csv"
# fname = "wb_metadata-2021-03-27 19:17:44.764136.csv"
# wb_metadata = pd.read_csv(dir_manager.get_data_dir("corpus", "WB", fname))

# collection = mongodb.get_collection("test_nlp", "docs_metadata")
# errors_list = []
# no_url = []
# metadata_list = []
# for i, row in wb_metadata.iterrows():
#     if i % 10000 == 0:
#         print(i)

#     row["_id"] = row["id"]

#     if row["url_txt"] == "not to be displayed--":
#         print(row["id"])
#         no_url.append(row)
#         continue

#     try:
#         meta = json.loads(meta_type.make_metadata_model_from_nlp_schema(row.fillna("")).json())
#         meta["_id"] = meta["id"]
#         metadata_list.append(meta)
#     except:
#         errors_list.append(row)

# collection.insert_many(metadata_list)

In [ ]:
fname = "wb_metadata-2021-03-27 19:17:44.764136.csv"
wb_metadata = pd.read_csv(dir_manager.get_data_dir("corpus", "WB", fname))

collection = mongodb.get_collection("test_nlp", "docs_metadata")
errors_list = []
no_url = []
metadata_list = []
ids = set()
dup_ids = set()
for i, row in wb_metadata.iterrows():
    if i % 10000 == 0:
        print(i)
    row["_id"] = row["id"]
    if row["url_txt"] == "not to be displayed--":
        print(row["id"])
        no_url.append(row)
        continue
    try:
        meta = json.loads(meta_type.make_metadata_model_from_nlp_schema(row.fillna("")).json())
        if meta["path_original"] is None or "TXT_ORIG" in meta["path_original"]:
            meta["path_original"] = meta["path_original"]
        else:
            p = Path(meta["path_original"])
            meta["path_original"] = str(p.parent / "TXT_ORIG" / p.name)
        meta["_id"] = meta["id"]
        if meta["_id"] in ids:
            dup_ids.add(meta["_id"])
            continue
        metadata_list.append(meta)
        ids.add(meta["_id"])
    except:
        errors_list.append(row)
print(len(ids))
filter_obj = {"id": {"$in": list(ids)}}
del_res = collection.delete_many(filter_obj)
ins_res = collection.insert_many(metadata_list)

In [ ]:
def make_nlp_docs_from_docs_metadata(docs_metadata):
    root_path = Path(dir_manager.get_path_from_root())
    for ix, data in enumerate(docs_metadata):
        if ix and ix % 10000 == 0:
            print(ix)
        doc_path = root_path / data["path_original"]
        if not doc_path.exists():
            continue
        # create and save and article
        nlp_doc = elasticsearch.NLPDoc(meta={'id': data["_id"]}, **data)
        with open(doc_path, 'rb') as open_file:
            doc = open_file.read().decode('utf-8', errors='ignore')
            nlp_doc.body = doc
        nlp_doc.save()

In [ ]:
from wb_nlp.interfaces import elasticsearch

es = elasticsearch.get_client()
es.indices.refresh(index_name)
es.cat.count(index_name, params={"format": "json"})

# Notes and scratch

1. Copy raw data from sandbox server to gw1. Then copy to server7. Make sure the files are in the proper format <corpus_id>_<doc_id>.*
2. Extract covers of pdf.
3. Run filter by language script.
4. Set up config for the cleaning and the models. Then load the configs to db.
5. Define the scripts to run the models in model_runs/<model_name>/<model_name>-run-<run_number>.sh and also create the cleaning script (run_<model_name__cleaning.sh) for the intended model configuration.
6. Execute cleaning and traning: bash run_<model_name__cleaning.sh && bash model_runs/<model_name>/<model_name>-run-<run_number>.sh
7. Update permission of the data and models directories (inside the container).
8. Tar the needed files and models.
9. rsync to gw1, then rsync to sandbox/dev server.
10. copy files to local directories, i.e., data/ and models/.
11. Load the metadata to db.
12. Make a softlink from EN_TXT_ORIG to TXT_ORIG
13. Build the document vectors.
14. Dump the documents to elasticsearch.
15. Update the default model ids in app/app_kcp/src/config.js

##############################

https://stackoverflow.com/a/44683248

```FROM ubuntu:xenial-20170214
ARG UNAME=testuser
ARG UID=1000
ARG GID=1000
RUN groupadd -g $GID -o $UNAME
RUN useradd -m -u $UID -g $GID -o -s /bin/bash $UNAME
USER $UNAME
CMD /bin/bash


docker build --build-arg UID=$(id -u) --build-arg GID=$(id -g) \
  -f bb.dockerfile -t testimg .
```

##############################

```sudo docker-compose -f docker-compose.yml -f docker-compose.w0lxsnlp01.yml up -d --no-deps --build tika
sudo docker-compose -f docker-compose.yml -f docker-compose.w0lxsnlp01.yml up -d --no-deps --build redis
sudo docker-compose -f docker-compose.yml -f docker-compose.w0lxsnlp01.yml up -d --no-deps --build mongodb
sudo docker-compose -f docker-compose.yml -f docker-compose.w0lxsnlp01.yml up -d --no-deps --build milvus

sudo docker-compose -f docker-compose.yml -f docker-compose.w0lxsnlp01.yml up -d --no-deps --build nlp_api
sudo docker-compose -f docker-compose.yml -f docker-compose.w0lxsnlp01.yml up -d --no-deps --build app_kcp

Step 1: Copy nlp-metadata-wbes2474-20201007.json to data/raw
Step 2: Place sample data in data/raw/sample_data/TXT_ORIG

sudo docker exec -it wb_nlp_nlp_api_1 /bin/sh

run scripts

###################################
To copy data from sandbox to gw1

python ~/kcp-scripts/copy_raw_data.py

# Manually rsync data for ADB
rsync -avP w0lxsnlp01:/decfile2/Modeling/NLP/ihsn_scrapers/scrapers/adb/adb_files/full /Documentum/Aivin/corpus/ADB/
mv /Documentum/Aivin/corpus/ADB/full /Documentum/Aivin/corpus/ADB/PDF_ORIG


###################################
Generate wikipedia vocab
Download data from: https://dumps.wikimedia.org/enwiki/latest/
enwiki-latest-pages-articles.xml.bz2: https://dumps.wikimedia.org/enwiki/latest/enwiki-latest-pages-articles.xml.bz2

cd /data/wb536061/wikipedia-word-frequency
./gather_wordfreq.py ../wb_nlp/data/external/wikipedia/enwiki-latest-pages-articles.xml.bz2 > wordfreq-enwiki-latest-pages-articles.xml.bz2.txt


sudo /usr/local/bin/docker-compose -f docker-compose.yml -f docker-compose.server7.yml up -d --no-deps --build tika
sudo /usr/local/bin/docker-compose -f docker-compose.yml -f docker-compose.server7.yml up -d --no-deps --build redis
sudo /usr/local/bin/docker-compose -f docker-compose.yml -f docker-compose.server7.yml up -d --no-deps --build mongodb
sudo /usr/local/bin/docker-compose -f docker-compose.yml -f docker-compose.server7.yml up -d --no-deps --build milvus

sudo /usr/local/bin/docker-compose -f docker-compose.yml -f docker-compose.server7.yml up -d --no-deps --build nlp_api
sudo /usr/local/bin/docker-compose -f docker-compose.yml -f docker-compose.server7.yml up -d --no-deps --build app_kcp

sudo docker exec -it wb_nlp_nlp_api_1 /bin/bash

python -u ./scripts/cleaning/convert_pdf2txt_corpus.py --input-dir ./data/corpus --recursive -vv |& tee ./logs/convert_pdf2txt_corpus.py.log

bash run_word2vec_cleaning.sh && bash model_runs/word2vec/word2vec-run-01.sh

### Build a tar of word2vec model and WB data
tar -czf KCP_WB_DATA_MODELS.tar.gz data/corpus/WB/COVER data/corpus/WB/wb_metadata-2021-03-27\ 19\:17\:44.764136.csv data/corpus/WB/EN_TXT_ORIG data/corpus/cleaned/229abf370f281efa7c9f3c4ddc20159d/WB models/word2vec models/lda

from wb_nlp.models import word2vec_base
import logging

wvec_model = word2vec_base.Word2VecModel(
    model_config_id="702984027cfedde344961b8b9461bfd3",
    cleaning_config_id="229abf370f281efa7c9f3c4ddc20159d",
    model_run_info_id="854ae5f9cdda093265212c435d1ddfd4",
    raise_empty_doc_status=False,
    log_level=logging.DEBUG)

wvec_model.build_doc_vecs(pool_workers=22)


###################### WDI
import pandas as pd
import logging
from wb_nlp import dir_manager
from wb_nlp.models import word2vec_base


wvec_model = word2vec_base.Word2VecModel(
    model_config_id="702984027cfedde344961b8b9461bfd3",
    cleaning_config_id="229abf370f281efa7c9f3c4ddc20159d",
    model_run_info_id="854ae5f9cdda093265212c435d1ddfd4",
    raise_empty_doc_status=False,
    log_level=logging.DEBUG)

wdi_df = pd.read_csv(dir_manager.get_data_dir(
    "preprocessed", "timeseries", "wdi_time_series_metadata.csv"))

wdi_df["text"] = wdi_df["txt_meta"].map(wvec_model.clean_text)

wdi_df["vector"] = wdi_df.apply(
    lambda x: wvec_model.process_doc(x)["doc_vec"], axis=1)

wdi_df.to_pickle(
    f"/workspace/models/wdi/wdi_time_series_metadata-{wvec_model.model_id}.pickle")```
